# ICT3104 Project Team 12

# 💃 Setup

## 🕺 External Repositories
The tool makes use of additional repositories such as FollowYourPose and MMPose. Therefore, these repositories must be cloned and set up.

Additionally, since the tool utilises Google Colab, our repository must also be cloned.  

### 👯 Working Repository

In [1]:
%cd /content
!git clone https://github.com/Sitrof9/ict3104-team12-2023.git
%cd ict3104-team12-2023
!pip install -r requirements.txt
!pip install -r requirements-fyp.txt


/content
Cloning into 'ict3104-team12-2023'...
remote: Enumerating objects: 444, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 444 (delta 69), reused 62 (delta 50), pack-reused 354
Receiving objects: 100% (444/444), 211.42 MiB | 36.98 MiB/s, done.
Resolving deltas: 100% (166/166), done.
/content/ict3104-team12-2023
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

### 👯 Follow Your **Pose**

#### 🪩 Install Environment

In [2]:
%cd /content
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1
!python --version
!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip

print('Git clone project and install requirements...')
!git clone https://github.com/mayuelala/FollowYourPose.git

/content
update-alternatives: error: alternative path /usr/bin/python3.8 doesn't exist
update-alternatives: error: alternative path /usr/bin/python3.9 doesn't exist
Python 3.10.12
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [589 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,278 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packag

In [3]:
!cd /content/FollowYourPose
!export PYTHONPATH=/content/FollowYourPose:$PYTHONPATH
# %pip install -r requirements.txt
!python -m pip install -q -U --pre triton
!apt update
!python -m pip install -q diffusers==0.11.1 torch==1.13.1 transformers==4.26.0 bitsandbytes==0.35.4 \
imageio-ffmpeg xformers==0.0.16  --extra-index-url https://download.pytorch.org/whl/cu113

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.1 MB/s eta 0:00:00
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
22 packages can be upgraded. Run 'apt list --upgradable' to see them.
     ━━━━━━━━━━━━━

#### 🪩 Weight

In [4]:
%cd /content/FollowYourPose/
%mkdir checkpoints
!git lfs install
!git clone https://huggingface.co/YueMafighting/FollowYourPose_v1
%mv /content/FollowYourPose/FollowYourPose_v1/* /content/FollowYourPose/checkpoints/
%rm -rf FollowYourPose_v1

/content/FollowYourPose
Updated git hooks.
Git LFS initialized.
Cloning into 'FollowYourPose_v1'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), 584.62 KiB | 4.60 MiB/s, done.
Filtering content: 100% (9/9), 9.75 GiB | 52.54 MiB/s, done.


### 👯 MMPose

In [5]:
%cd /content
!git clone https://github.com/open-mmlab/mmpose.git
%cd mmpose
!pip install -r requirements.txt
!pip install -v -e .
!pip install --upgrade bitsandbytes

/content
Cloning into 'mmpose'...
remote: Enumerating objects: 30276, done.
remote: Counting objects: 100% (1052/1052), done.
remote: Compressing objects: 100% (639/639), done.
remote: Total 30276 (delta 517), reused 812 (delta 404), pack-reused 29224
Receiving objects: 100% (30276/30276), 53.58 MiB | 25.10 MiB/s, done.
Resolving deltas: 100% (21325/21325), done.
/content/mmpose
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.0/436.0 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 12.4 MB/s eta 0:00:00
     ━━━

## 🕺 Libraries
These libraries are required to run the tool.

### 👯 Installation

In [6]:
!pip install ruamel.yaml
!pip install tqdm
!pip install imageio
!pip install torch
!pip install moviepy
!pip install -U nvidia-cudnn-cu11
!pip install openmim
!pip install accelerate
!pip install nvidia-cublas-cu11
!pip install nvidia-cuda-nvrtc-cu11
!pip install nvidia-cuda-runtime-cu11
!pip install nvidia-cudnn-cu11==8.9.4.25
!pip install omegaconf
!pip install PyYAML
!mim install mmengine
!mim install "mmcv>=2.0.1"
!mim install "mmdet>=3.1.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 696.8/696.8 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cudnn-cu11
    Found existing installation: nvidia-cudnn-cu11 8.5.0.96
    Uninstalling nvidia-cudnn-cu11-8.5.0.96:
      Successfully uninstalled nvidia-cudnn-cu11-8.5.0.96
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 1.13.1 requires nvidia-cudnn-cu11==8.5.0.96; platform_system == "Linux", but you have nvidia-cudnn-cu11 8.9.5.30 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 34.7 MB/s e

  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Attempting uninstall: nvidia-cudnn-cu11
    Found existing installation: nvidia-cudnn-cu11 8.9.5.30
    Uninstalling nvidia-cudnn-cu11-8.9.5.30:
      Successfully uninstalled nvidia-cudnn-cu11-8.9.5.30
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.3/719.3 MB 1.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cudnn-cu11
    Found existing installation: nvidia-cudnn-cu11 8.5.0.96
    Uninstalling nvidia-cudnn-cu11-8.5.0.96:
      Successfully uninstalled nvidia-cudnn-cu11-8.5.0.96
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 1.13.1 requires nvidia-cudnn-cu11==8.5.0.96; platform_system == "Linux", but you have nvidia-cudnn-cu11 8.9.4.25 which is incompatible.
Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch1.13.0/index.html
     ━━━━━━━━━━━━

### 👯 Import

In [7]:
# Import libraries
import os
from os import walk
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, HBox
from IPython.display import display, clear_output, Video, HTML
import yaml
import ruamel.yaml
import subprocess
import shutil
import glob
import re
import datetime
import time
from time import sleep
import csv
from base64 import b64encode
import torch
from moviepy.editor import *
import moviepy.editor as mpe
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from tqdm import tqdm
from datetime import datetime
import cv2
from skimage.metrics import structural_similarity as compare_ssim
from skimage.metrics import mean_squared_error
import numpy as np


# 💃 Data Exploration
This section lets you select and view the videos from the data/input_files/training_videos folder, allowing you to verify if you have chosen the correct video. To view the videos, select an option from the dropdown video and wait for the video to be displayed.<br>

Other tools available
*   Video Resizer
*   Video Converter

To import your own videos, open the file explorer and place your videos into data/input_videos/training_videos.



## 🕺 Video Preview

In [ ]:
%cd /content
# Define folder path of training videos
main_file_path = f"./ict3104-team12-2023/data/input_files/"

@interact
def select_subfolder(video_folder=os.listdir(main_file_path)):
    video_folder_path = f"{main_file_path}{video_folder}/"

    @interact
    def show_video(file=os.listdir(video_folder_path)):
        if(".mp4" not in file):
            print("Please choose a file with .mp4 format.")
            return
        video_path = f"{video_folder_path}{file}"
        video_display = Video(video_path, width=512, height=512, embed=True)
        display(video_display)

/content


interactive(children=(Dropdown(description='video_folder', options=('training_videos', 'pose_examples'), value…

## 🕺 Video Resizer

In [ ]:
%cd /content
# To resize videos
def clip_resizer(clip_path, dest_path, height, width):
  clip = mpe.VideoFileClip(clip_path)
  clip_resized = clip.resize((height, width))
  clip_resized.write_videofile(dest_path)

# Create button for showing video
resizeButton = widgets.Button(description="Resize Video")

# Unregister previos button's click event
def clear_button_clicks(button):
    button._click_handlers.callbacks = []

# Define folder path of training videos
main_file_path = f"./ict3104-team12-2023/data/input_files/"

@interact
def select_subfolder(video_folder=os.listdir(main_file_path)):
    video_folder_path = f"{main_file_path}{video_folder}/"

    @interact
    def resize_video(file=os.listdir(video_folder_path)):

        def on_button_click(b):
            if(".mp4" not in file):
                print("Please choose a file with .mp4 format.")
                return

            video_path = f"{video_folder_path}{file}"
            new_file_name = f"{file[:len(file)-4]}_resized.mp4"
            destination_path = f"{video_folder_path}{new_file_name}"
            clip_resizer(video_path, destination_path, 512, 512)

        # Clear previous button clicks
        clear_button_clicks(resizeButton)

        # Attach click event to the button
        resizeButton.on_click(on_button_click)

        # Display the button
        display(resizeButton)


/content


interactive(children=(Dropdown(description='video_folder', options=('training_videos', 'pose_examples'), value…

chunk:   0%|          | 1/1332 [02:01<44:56:22, 121.55s/it, now=None]

Moviepy - Building video ./ict3104-team12-2023/data/input_files/training_videos/Wa_S5D1_fC4.aviWa_S5D1_fC4_resized.mp4.
MoviePy - Writing audio in Wa_S5D1_fC4.aviWa_S5D1_fC4_resizedTEMP_MPY_wvf_snd.mp3



chunk:   0%|          | 1/1332 [02:03<45:35:16, 123.30s/it, now=None]

...

Moviepy - Building video ./ict3104-team12-2023/data/input_files/training_videos/Wa_S5D1_fC4.aviWa_S5D1_fC4_resized.mp4.
MoviePy - Writing audio in Wa_S5D1_fC4.aviWa_S5D1_fC4_resizedTEMP_MPY_wvf_snd.mp3



chunk:   0%|          | 1/1332 [02:48<62:08:58, 168.10s/it, now=None]

MoviePy - Done.
Moviepy - Writing video ./ict3104-team12-2023/data/input_files/training_videos/Wa_S5D1_fC4.aviWa_S5D1_fC4_resized.mp4




chunk:   0%|          | 1/1332 [03:25<75:52:16, 205.21s/it, now=None]

Moviepy - Done !
Moviepy - video ready ./ict3104-team12-2023/data/input_files/training_videos/Wa_S5D1_fC4.aviWa_S5D1_fC4_resized.mp4


## 🕺Video Converter


In [ ]:
%cd /content

# Create button for showing video
convertButton = widgets.Button(description="Convert Video")

# Unregister previos button's click event
def clear_button_clicks(button):
    button._click_handlers.callbacks = []

# Define folder path of training videos
main_file_path = f"./ict3104-team12-2023/data/input_files/"

@interact
def select_subfolder(video_folder=os.listdir(main_file_path)):
    video_folder_path = f"{main_file_path}{video_folder}/"

    @interact
    def convert_video(file=os.listdir(video_folder_path)):

        def on_button_click(b):
            if(".mp4" in file):
                print("File is already in .mp4 format.")
                return

            try:
              video_path = f"{video_folder_path}{file}"
              new_file_name = f"{file[:len(file)-4]}.mp4"
              destination_path = f"{video_folder_path}{new_file_name}"

              clip = mpe.VideoFileClip(video_path)
              clip_resized = clip.resize((512, 512))
              clip_resized.write_videofile(destination_path)
            except:
              print("There was an issue with the file.")

        # Clear previous button clicks
        clear_button_clicks(convertButton)

        # Attach click event to the button
        convertButton.on_click(on_button_click)

        # Display the button
        display(convertButton)


# 💃 Inference

## 🕺Skeleton Generation
This section lets you select a video from the subfolders of the input_files folder and generates a skeleton based on that. The video will be saved under data/input_files/vis_results.

In [ ]:
%cd /content
# Define folder path of training videos
main_file_path = f"./ict3104-team12-2023/data/input_files/"

# Create button for showing video
button = widgets.Button(description="Generate Skeleton")

# Unregister previos button's click event
def clear_button_clicks(button):
    button._click_handlers.callbacks = []

@interact
def select_subfolder(video_folder=os.listdir(main_file_path)):
    video_folder_path = f"{main_file_path}{video_folder}/"

    @interact
    def show_video(file=os.listdir(video_folder_path)):

        def on_button_click(b):
            if(".mp4" not in file):
                print("Please choose a file with .mp4 format.")
                return
            video_path = video_folder_path + file
            video_path = video_path[2:]
            file_name = file[:len(file)-4]

            # Get the current date and time
            current_time = datetime.now()
            formatted_time = current_time.strftime("%Y%m%d_%H%M%S")
            new_file_name = f"{file_name}_{formatted_time}"

            output_path = f"{main_file_path}vis_results/{new_file_name}"
            print(output_path)

            print(os.getcwd())
            inference_path = "./mmpose/demo/inferencer_demo.py"

            shell_command = f"python {inference_path} {video_path} --pose2d human --black-background --thickness 5 --vis-out-dir {output_path}"
            # subprocess.run(shell_command, shell=True)
            with subprocess.Popen(shell_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True) as proc:
              for line in iter(proc.stdout.readline, ''):
                  print("stdout: " + line.strip())

              for line in iter(proc.stderr.readline, ''):
                  print("stderr: " + line.strip())
            print("process done")

            # Move file from subfolder into folder, rename file, and delete subfolder
            # Move file
            source = f"{output_path}/{file}"
            renamed_source = f"{output_path}/{new_file_name}.mp4"
            print(f"source: {source}")
            print(f"renamed_source: {renamed_source}")
            destination = f"{main_file_path}/vis_results"
            os.rename(source, renamed_source)
            shutil.move(renamed_source, destination)

            # Remove subfolder
            try:
              os.rmdir(output_path)
            except OSError:
               shutil.rmtree(output_path)


        # Clear previous button clicks
        clear_button_clicks(button)

        # Attach click event to the button
        button.on_click(on_button_click)

        # Display the button
        display(button)

/content


interactive(children=(Dropdown(description='video_folder', options=('pose_examples', 'training_videos'), value…

## 🕺Inference Video Generation

This section lets you interactively select a skeleton video and configure prompts to run an inference process. The results are displayed in a grid layout, with each row containing a combined video clip and the corresponding prompt.

In [43]:
%cd /content
video_path = "./ict3104-team12-2023/data/input_files/pose_examples/"

video_files = [f for f in os.listdir(video_path) if f.endswith((".mov", ".mp4"))]

video_dropdown = widgets.Dropdown(
    options=video_files,
    description="Select a video: ", style = {"description_width": "initial","width": "30%"},
    layout=widgets.Layout(width="30%"),
)

# Variables to store the selected videos
selected_video = None
video_file_path = None

# Display the selected videos and store the selection in the variables
def display_video(selected):
    global selected_video, video_file_path
    selected_video = selected
    video_file_path = os.path.join(video_path, selected)

    # Update the print statement
    print("Selected video: ", selected_video, end='\n\n')

interact(display_video, selected=video_dropdown)

# Create a text input widget for pretrained_model_path
pretrained_model_path_input = widgets.Text(
    value="./checkpoints/stable-diffusion-v1-4",
    description="Pretrained Model Path:", style = {"description_width": "initial","width": "30%"},
    layout=widgets.Layout(width="30%"),
)

# Create a text input widget for yaml_file_path
yaml_file_path_input = widgets.Text(
    value="./ict3104-team12-2023/configs/pose_sample.yaml",
    description="YAML File Path:", style = {"description_width": "initial","width": "30%"},
    layout=widgets.Layout(width="30%"),
)
yaml_file_path = yaml_file_path_input.value

with open(yaml_file_path, 'r') as file:
    try:
        data = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

pretrained_model_path = data['pretrained_model_path']
validation_data = data['validation_data']
prompts = validation_data['prompts']

# Create a text input widget for prompts
prompts_input = widgets.Textarea(
    value = "\n".join(str(i) for i in (validation_data['prompts'])),
    description = "Prompts:", style = {"description_width": "initial","width": "30%"},
    rows = 4,
    layout=widgets.Layout(width="30%"),
)

update_config_button = widgets.Button(description="Update Config")
output_widget = widgets.Output()

def update_config(button_click):
    updated_path = pretrained_model_path_input.value
    updated_prompts = prompts_input.value.split("\n")

    data['pretrained_model_path'] = updated_path
    if updated_prompts[len(updated_prompts)-1] == "":
      data['validation_data']['prompts'] = updated_prompts[:len(updated_prompts)-1]
    else:
      data['validation_data']['prompts'] = updated_prompts

    # Set video length to 8
    data['validation_data']['video_length'] = 8

    # Write the updated content back to the file
    with open(yaml_file_path, 'w') as yaml_file:
        yaml.dump(data, yaml_file, sort_keys=False)

    # Display the updated YAML content in the output widget
    with output_widget:
        clear_output()
        print(yaml.dump(data, sort_keys=False))

    os.chdir("/content/FollowYourPose")
    display(run_inference_button)

update_config_button.on_click(update_config)

# Display input fields
display(pretrained_model_path_input)
display(yaml_file_path_input)
display(prompts_input)

# Preview
display(update_config_button, output_widget)

run_inference_button = widgets.Button(description="Run Inference")

def run_inference(button_click):
  #get all variables needed
  #yaml config used path
  yaml_config_path = "../"+yaml_file_path #YAML path extracted from selection

  #skeleton path
  initial_skeleton_path = "../"+video_file_path #skeleton path extracted from selection
  actual_skeleton_path = initial_skeleton_path[0:-4] + ".mp4"

  #convert skeleton vid to mp4
  if not os.path.isfile(actual_skeleton_path):
      !ffmpeg -i {initial_skeleton_path} {actual_skeleton_path}

  #Execute Inference to produce results
  result_array = !TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config={yaml_config_path} --skeleton_path={initial_skeleton_path}
  #find result path
  for i in range(len(result_array)-1,-1,-1):
    if "Saved samples to" in result_array[i]:
      result_location = result_array[i].split("to ")
      break

  result_path = result_location[1][0:-4] #result path extracted from finished result

  #Build and Combine Files
  for (dirpath, dirnames, filenames) in walk(result_path):
      break

  #Temporarily store snippet of video here
  new_skeleton_path = "./Dummy1.mp4" #change this to reference a possible path
  edited_video_path = "./Dummy.mp4"  #change this to reference a possible path

  #result video path
  outputFile = "./output/samples/"
  #Edit video based on inputs
  ffmpeg_extract_subclip(actual_skeleton_path,0,0.544,targetname=edited_video_path) #cut's video
  !ffmpeg -i {edited_video_path} -vf "setpts=1.65*PTS" {new_skeleton_path} #slow video down

  clear_output(wait=True)

  #initialize list to save all result videos
  all_Results = []

  #display all results
  for i in filenames:
    tempOutputPath = outputFile+i[0:-3]+" Combined.mp4"
    #produce superImpose Clip
    result_clip = mpe.VideoFileClip(result_path+"/"+i, target_resolution=(1080,1920))

    overlay_clip = mpe.VideoFileClip(new_skeleton_path, has_mask=True, target_resolution=(1080,1920))
    masked_clip = mpe.vfx.mask_color(overlay_clip,color=[0,0,0])

    superImpose_clip = mpe.CompositeVideoClip([result_clip,masked_clip])
    combined_clip = mpe.clips_array([[result_clip,overlay_clip,superImpose_clip]])
    combined_clip = combined_clip.resize((1680,560))
    combined_clip.write_videofile(
        tempOutputPath,
        remove_temp=True,
        codec="libx264",
        audio_codec="aac",
        threads=6
    )

    all_Results.append(tempOutputPath)

  #delete Temporary files
  !rm {edited_video_path}
  !rm {new_skeleton_path}

  #display results
  for i in all_Results:
    #Formatting
    grid = widgets.GridspecLayout(2,1) #(i,j) i= number of columns, j= number of rows

    container = widgets.VBox([grid], layout = widgets.Layout(
        height = '650px',
        width='1690px',
        border='3px solid black',
        justify_content = 'center',
        align_items='center'
    ))

    #show skeleton video
    out = widgets.Output()
    with out:
        display(Video(i,embed=True,html_attributes="loop autoplay"))
    grid[0,0] = out

    #display prompt
    out = widgets.Output()
    with out:
        display(HTML("<center><h1>"+i[0:-14].split("/")[3]+"</h1></center>"))
    grid[1,:] = out

    display(container)

run_inference_button.on_click(run_inference)

/content


interactive(children=(Dropdown(description='Select a video: ', layout=Layout(width='30%'), options=('vis_ikun_…

Text(value='./checkpoints/stable-diffusion-v1-4', description='Pretrained Model Path:', layout=Layout(width='3…

Text(value='./ict3104-team12-2023/configs/pose_sample.yaml', description='YAML File Path:', layout=Layout(widt…

Textarea(value='Iron man on the beach', description='Prompts:', layout=Layout(width='30%'), rows=4, style=Desc…

Button(description='Update Config', style=ButtonStyle())

Output()

Button(description='Run Inference', style=ButtonStyle())

In [ ]:
# Paths to your real and generated videos
real_video_path = "/content/ict3104-team12-2023/data/input_files/training_videos/6R7QW.mp4" #Testing purposes
generated_video_path = "/content/ict3104-team12-2023/data/input_files/training_videos/CCCUJ.mp4" #Testing purposes

# Open the video files
real_video = cv2.VideoCapture(real_video_path)
generated_video = cv2.VideoCapture(generated_video_path)

# Initialize variables to store cumulative scores
total_ssim_score = 0.0
total_mse_score = 0.0
frame_count = 0

# Loop through the frames of the videos
while True:
    # Read frames from both videos
    ret_real, frame_real = real_video.read()
    ret_generated, frame_generated = generated_video.read()

    # Check if both videos have more frames
    if not ret_real or not ret_generated:
        break

    # Ensure both frames have the same dimensions (resize if necessary)
    if frame_real.shape != frame_generated.shape:
        frame_generated = cv2.resize(frame_generated, (frame_real.shape[1], frame_real.shape[0]))

    # Convert frames to grayscale for SSIM calculation (assuming grayscale video)
    frame_real_gray = cv2.cvtColor(frame_real, cv2.COLOR_BGR2GRAY)
    frame_generated_gray = cv2.cvtColor(frame_generated, cv2.COLOR_BGR2GRAY)

    # Calculate SSIM and MSE for the frames
    ssim_score = compare_ssim(frame_real_gray, frame_generated_gray)
    mse_score = mean_squared_error(frame_real, frame_generated)

    # Update cumulative scores
    total_ssim_score += ssim_score
    total_mse_score += mse_score
    frame_count += 1

# Calculate average scores
average_ssim_score = total_ssim_score / frame_count
average_mse_score = total_mse_score / frame_count

# Close the video files
real_video.release()
generated_video.release()

# Print the average scores
print(f"Average SSIM Score: {average_ssim_score}")
print(f"Average MSE Score: {average_mse_score}")

# 💃 Training

This section imports all the necessary configuration changes for the **3 files: pose_train.yaml, hdvila.py and train_followyourpose.py**, from the ict3104-team12-2023 directory to replace the one on the FollowYourPose directory, so that we will be able to run the training of the model under lower configurations (due to Colab limitations)

In [ ]:
# cd into FollowYourPose as we will be mainly using this directory for training
%cd /content/FollowYourPose

# Import the pose_train.yaml from the ict3104-team12-2023 directory to replace the one on FollowYourPose
# The first difference is that max_train_steps has been decreased from 5000 to 5 (due to Colab limitations)
# The second difference is that the training data has been rescaled to 256 x 256 (due to Colab limitations)

# Define source path for pose_train.yaml
pose_train_source_file = "/content/ict3104-team12-2023/training_config_files/pose_train.yaml"

# Define destination path for pose_train.yaml
pose_train_destination_file = "/content/FollowYourPose/configs/pose_train.yaml"

# Copy "pose_train.yaml" to the destination directory
shutil.copy(pose_train_source_file, pose_train_destination_file)
print(f"pose_train.yaml has been copied to: {pose_train_destination_file}")


# Import the hdvila.py from ict3104-team12-2023 directory to replace the one on FollowYourPose
# The first difference is that the data directory is set to '/content/FollowYourPose/followyourpose/data' to find the 'caption_rm2048_train.csv'
# The second difference is that the second element of 'sample[1]', which contains a list of captions associated with that clip, has been changed to '[0]' (since there is only 1 caption associated to each video)

# Define source path for hdvila.py
hdvila_source_file = "/content/ict3104-team12-2023/training_config_files/hdvila.py"

# Define destination path for hdvila.py
hdvila_destination_file = "/content/FollowYourPose/followyourpose/data/hdvila.py"

# Copy "hdvila.py" to the destination directory
shutil.copy(hdvila_source_file, hdvila_destination_file)
print(f"hdvila.py has been copied to: {hdvila_destination_file}")

# Import the train_followyourpose.py from ict3104-team12-2023 directory to replace the one on FollowYourPose
# The difference is that the training command has an added argument of the skeleton_path (so that we will able to iteratively run the training command on different skeletons across different directory paths)

# Define source path for train_followyourpose.py
train_followyourpose_source_file = "/content/ict3104-team12-2023/training_config_files/train_followyourpose.py"

# Define destination path for train_followyourpose.py
train_followyourpose_destination_file = "/content/FollowYourPose/train_followyourpose.py"

# Copy "train_followyourpose.py" to the destination directory
shutil.copy(train_followyourpose_source_file, train_followyourpose_destination_file)
print(f"train_followyourpose.py has been copied to: {train_followyourpose_destination_file}")

This section creates the required file directories preset in the hdvila.py **(/part_{part_id}/video_clips)**. The MMPose skeleton videos generated using the Charades dataset are then copied into the directories respectively.

A CSV **(caption_rm2048_train.csv)** file is also generated and populated with the data of each individual video clip storage directory and video clip name as well a caption (that will be input by the user) which will be used to train the model.

In [ ]:
# Define source path for the MMPose skeleton videos
pose_examples_source_directory = '/content/ict3104-team12-2023/data/input_files/pose_examples'
#pose_examples_source_directory = '/content/ict3104-team12-2023/data/input_files/vis_results'

# Define destination path for the MMPose skeleton videos
pose_examples_destination_directory = '/content/FollowYourPose/followyourpose/data'

# Iterate through the source path directory and count the number of MMPose skeleton videos
video_files = [f for f in os.listdir(pose_examples_source_directory) if f.endswith('.mov')]
num_videos = len(video_files)

# Based on the number of MMPose skeleton videos, create the part_id folders
for part_id in range(1, num_videos + 1):
    part_folder = os.path.join(pose_examples_destination_directory, f'part_{part_id}')
    os.makedirs(part_folder, exist_ok=True)

# Copy each MMPose skeleton video into a separate 'part_/video_clips' folder (this is based on the required file directory preset in hdvila.py)
for i, video_file in enumerate(video_files[:10]):
    part_id = i + 1
    part_folder = os.path.join(pose_examples_destination_directory, f'part_{part_id}')
    video_clips_folder = os.path.join(part_folder, 'video_clips')
    os.makedirs(video_clips_folder, exist_ok=True)

    source_path = os.path.join(pose_examples_source_directory, video_file)
    video_count = 1

    while True:
        destination_path = os.path.join(video_clips_folder, f'{video_count:04}.mp4')
        if not os.path.exists(destination_path):
            break
        video_count += 1

    shutil.copy(source_path, destination_path)

# Create the 'caption_rm2048_train.csv' (which is required for the model to read and identify each caption to each MMPose skeleton video)
csv_filename = 'caption_rm2048_train.csv'
csv_path = os.path.join(pose_examples_destination_directory, csv_filename)

with open(csv_path, 'w', newline='') as csv_file:
    fieldnames = ['part_id', 'clip_id', 'caption']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()

    # Iterate through the 'part_/video_clips' folders and write each video clip found as a record into a row into the 'caption_rm2048_train.csv' (with each data populated under their respective columns)
    for part_id in range(1, num_videos + 1):
        part_folder = os.path.join(pose_examples_destination_directory, f'part_{part_id}')
        video_clips_folder = os.path.join(part_folder, 'video_clips')

        for clip_id, video_file in enumerate(os.listdir(video_clips_folder), start=1):
            video_path = os.path.join(video_clips_folder, f'{clip_id:04}.mp4')
            clip_id = f'{clip_id:04}'
            print(f"Please enter a new caption for {video_path} (press Enter after keying in):\n")
            caption = input()
            print("\n")

            writer.writerow({'part_id': part_id, 'clip_id': clip_id, 'caption': caption})

print(f"\n{csv_filename} has been created at: {csv_path}")

This section executes the training command and saves all the newly trained model under **content/ict3104-team12-2023/data/models/trained_model**.

In [ ]:
# Define the 'caption_rm2048_train.csv' path
csv_path = '/content/FollowYourPose/followyourpose/data/caption_rm2048_train.csv'

# Define the base path for video clips
base_path = "/content/FollowYourPose/followyourpose/data"

# Define the destination folder for config.yaml files
destination_folder = '/content/ict3104-team12-2023/data/models/trained_model'

# Create a progress bar widget
progress_bar = widgets.FloatProgress(
    value=0.0,
    min=0.0,
    max=1.0,
    description='Progress:',
    bar_style='info',
)

# Create an HTML widget for displaying the percentage
percentage_label = widgets.HTML(value="0.00%")

# Create an HBox widget to place the progress bar and percentage together
progress_container = HBox([progress_bar, percentage_label])

# Display the progress bar and percentage label
display(progress_container)

# Record the start time
start_time = time.time()

# Calculate the total number of iterations (videos) to process
total_videos = sum(len(os.listdir(os.path.join(base_path, part_dir, "video_clips"))) for part_dir in os.listdir(base_path) if part_dir.startswith("part_"))

# Initialize a counter for processed videos
processed_videos = 0

# Create an empty dictionary to store config.yaml file paths
config_files = {}

# Load CSV data into a list of dictionaries
csv_data = []
with open(csv_path, 'r', newline='') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        csv_data.append(row)

def get_part_id(directory_name):
    # Extract the part ID from the directory name
    return directory_name.replace("part_", "")

def process_video(video_info, video_path, part_id, clip_id_with_zeros, destination_folder):
    print(f"Part ID: {part_id}")
    print(f"Clip ID: {clip_id_with_zeros}")
    print(f"Found video info: {video_info}")
    print(f"Video path: {video_path}")
    print(f"Processing video for Clip ID: {clip_id_with_zeros}")

    # Load the pose_train.yaml
    with open('/content/FollowYourPose/configs/pose_train.yaml', 'r') as config_file:
        config = yaml.safe_load(config_file)

    # Update the skeleton_path in the config dictionary
    config['skeleton_path'] = f"./followyourpose/data/part_{part_id}/video_clips/{clip_id_with_zeros}.mp4"

    # Save the updated config to a new file for each clip
    config_file_path = os.path.join(destination_folder, f"config_{clip_id_with_zeros}_{part_id}.yaml")
    with open(config_file_path, 'w') as new_config_file:
        yaml.dump(config, new_config_file, default_flow_style=False)

    # Execute the accelerate command on the video
    command = f"TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch train_followyourpose.py --config='{config_file_path}' --skeleton_path='{video_path}'"
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    #print(result.stdout.decode())
    #print(result.stderr.decode())
    print(f"Finished processing video: {video_path}")

    # Delete the generated config YAML file
    os.remove(config_file_path)

    # Copy the entire output folder to the destination folder
    source_output_folder = "/content/FollowYourPose/output"
    inputdestination = input("\nEnter the new model name (press Enter after keying in):\n")
    destination_output_folder = os.path.join(destination_folder, f"{inputdestination}")
    shutil.copytree(source_output_folder, destination_output_folder)

    if result.returncode == 0:
        print(f"\nFinished processing video for Clip ID: {clip_id_with_zeros}")
    else:
        print(f"Error processing video for Clip ID: {clip_id_with_zeros}")


# Iterate through videos and update the progress bar
for part_dir in os.listdir(base_path):
    if part_dir.startswith("part_"):
        part_path = os.path.join(base_path, part_dir, "video_clips")
        part_id = get_part_id(part_dir)

        if os.path.exists(part_path):
            clip_counter = 0

            for clip_file in os.listdir(part_path):
                # Use the 'clip_id' from CSV data directly
                clip_id = clip_file.split('.')[0]

                # Check if the clip_id (file name) exists in the CSV data
                for row in csv_data:
                    if row['clip_id'] == clip_id and row['part_id'] == part_id:
                        video_info = row
                        # Construct the video path based on the clip_file and the base path
                        video_path = os.path.join(part_path, clip_file)

                        if os.path.exists(video_path):
                            clip_counter += 1
                            clip_id_with_zeros = str(clip_counter).zfill(4)

                            # Call the process_video function to handle video processing
                            process_video(video_info, video_path, part_id, clip_id_with_zeros, destination_folder)

                            # Update the progress bar
                            processed_videos += 1
                            progress_percentage = (processed_videos / total_videos) * 100
                            progress_bar.value = processed_videos / total_videos

                            # Update the percentage label
                            percentage_label.value = f'{progress_percentage:.2f}%'


# Check if it's the last iteration and remove the "output" folder
if processed_videos == total_videos:
    output_folder_path = "/content/FollowYourPose/output"
    shutil.rmtree(output_folder_path)

# Record the end time
end_time = time.time()

# Calculate the total runtime
total_runtime = end_time - start_time

# Display the total runtime
print(f"\nTotal Runtime: {total_runtime} seconds")

# Display a completion message
display(HTML('<h2>Processing completed!</h2>'))

This section saves the most optimal configuration (based on latest timestamp) under **content/ict3104-team12-2023/data/models/final_model**.

In [ ]:
# Change to the source directory
source_dir = "/content/ict3104-team12-2023/data/models/trained_model"
os.chdir(source_dir)

# Find all the subdirectories in the source directory
subdirectories = [d for d in os.listdir() if os.path.isdir(d)]

if subdirectories:
    # Sort the subdirectories by modification time to get the latest one
    latest_subdirectory = max(subdirectories, key=lambda d: os.path.getmtime(os.path.join(source_dir, d)))

    # Define the destination directory
    destination_dir = "/content/ict3104-team12-2023/data/models/final_model"

    # Check if the destination directory exists, create it if it doesn't
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Get the current timestamp in the format yyyymmdd_hhmmss
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Create the new directory name with the timestamp
    new_directory_name = f"{latest_subdirectory}"

    # Combine the destination directory and the new directory name
    new_destination_directory = os.path.join(destination_dir, new_directory_name)

    # Move the latest subdirectory to the destination directory with the new directory name
    shutil.move(os.path.join(source_dir, latest_subdirectory), new_destination_directory)
    print(f"Latest output folder renamed and moved to '{new_destination_directory}'.")
else:
    print("No output folders found in the source directory.")

# 💃 Testing

This section lets you interactively select multiple skeleton videos and configure prompts to run an inference process. The results are displayed in a grid layout, with each row containing a combined video clip and the corresponding prompt to allow for testing purposes

In [44]:
%cd /content
video_path = "./ict3104-team12-2023/data/input_files/pose_examples/"

video_files = [f for f in os.listdir(video_path) if f.endswith((".mov",".mp4"))]

video_dropdown = widgets.SelectMultiple(
    options=video_files,
    # Multiple values can be selected with shift or ctrl pressed and mouse clicks.
    description="Select videos: ", style = {"description_width": "initial","width": "30%"},
    layout=widgets.Layout(width="30%"),
)

# Variables to store the selected videos
selected_videos = []
video_file_paths = []

# Display the selected videos and store the selection in the variables
def display_videos(selected):
    global selected_videos, video_file_paths
    selected_videos = selected
    video_file_paths = [os.path.join(video_path, video) for video in selected]

    # Update the print statement
    print("Selected videos: ", selected_videos, end='\n\n')

interact(display_videos, selected=video_dropdown)

# Create a text input widget for pretrained_model_path
pretrained_model_path_input = widgets.Text(
    value="./checkpoints/stable-diffusion-v1-4",
    description="Pretrained Model Path:", style = {"description_width": "initial","width": "30%"},
    layout=widgets.Layout(width="30%"),
)

# Create a text input widget for yaml_file_path
yaml_file_path_input = widgets.Text(
    value="./ict3104-team12-2023/configs/pose_sample.yaml",
    description="YAML File Path:", style = {"description_width": "initial","width": "30%"},
    layout=widgets.Layout(width="30%"),
)
yaml_file_path = yaml_file_path_input.value

with open(yaml_file_path, 'r') as file:
    try:
        data = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

pretrained_model_path = data['pretrained_model_path']
validation_data = data['validation_data']
prompts = validation_data['prompts']

# Create a text input widget for prompts
prompts_input = widgets.Textarea(
    value = "\n".join(str(i) for i in (validation_data['prompts'])),
    description = "Prompts:", style = {"description_width": "initial","width": "30%"},
    rows = 4,
    layout=widgets.Layout(width="30%"),
)

update_config_button = widgets.Button(description="Update Config")
output_widget = widgets.Output()

def update_config(button_click):
    updated_path = pretrained_model_path_input.value
    updated_prompts = prompts_input.value.split("\n")

    data['pretrained_model_path'] = updated_path
    if updated_prompts[len(updated_prompts)-1] == "":
      data['validation_data']['prompts'] = updated_prompts[:len(updated_prompts)-1]
    else:
      data['validation_data']['prompts'] = updated_prompts

    # Set video length to 8
    data['validation_data']['video_length'] = 8

    # Write the updated content back to the file
    with open(yaml_file_path, 'w') as yaml_file:
        yaml.dump(data, yaml_file, sort_keys=False)

    # Display the updated YAML content in the output widget
    with output_widget:
        clear_output()
        print(yaml.dump(data, sort_keys=False))

    os.chdir("/content/FollowYourPose")
    display(run_inference_button)

update_config_button.on_click(update_config)

# Display input fields
display(pretrained_model_path_input)
display(yaml_file_path_input)
display(prompts_input)

# Preview
display(update_config_button, output_widget)

run_inference_button = widgets.Button(description="Run Inference")

def run_inference(button_click):
  #initialize list to save all result videos
  all_Results = []
  skeleton_paths = []
  result_paths = []
  #result video path
  outputFile = "./output/samples/"

  #get all variables needed
  #yaml config used path
  yaml_config_path = "../"+yaml_file_path #YAML path extracted from selection

  for i in range(len(video_file_paths)):
      #skeleton path
      initial_skeleton_path = "../"+video_file_paths[i] #skeleton path extracted from selection
      actual_skeleton_path = initial_skeleton_path[0:-4] + ".mp4"
      skeleton_paths.append(actual_skeleton_path)

      #convert skeleton vid to mp4
      if not os.path.isfile(actual_skeleton_path):
          !ffmpeg -i {initial_skeleton_path} {actual_skeleton_path}

      #Execute Inference to produce results
      result_array = !TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config={yaml_config_path} --skeleton_path={initial_skeleton_path}
      #find result path
      for i in range(len(result_array)-1,-1,-1):
        if "Saved samples to" in result_array[i]:
          result_location = result_array[i].split("to ")
          break

      result_paths.append(result_location[1][0:-4]) #result path extracted from finished result

  for i in range(len(result_paths)):
      #Build and Combine Files
      for (dirpath, dirnames, filenames) in walk(result_paths[i]):
          break

      #Temporarily store snippet of video here
      new_skeleton_path = "./Dummy1.mp4" #change this to reference a possible path
      edited_video_path = "./Dummy.mp4"  #change this to reference a possible path

      #Edit video based on inputs
      ffmpeg_extract_subclip(skeleton_paths[i],0,0.544,targetname=edited_video_path) #cut's video
      !ffmpeg -y -i {edited_video_path} -vf "setpts=1.65*PTS" {new_skeleton_path} #slow video down, always overwrites

      clear_output(wait=True)

      #make a directory to store different skeleton files
      #skeleton name
      only_skeleton_name = skeleton_paths[i].split("/")[len(skeleton_paths[i].split("/"))-1][0:-4]

      #check for existing repository
      if not os.path.isdir(outputFile+only_skeleton_name):
          os.mkdir(outputFile+only_skeleton_name)
      #display all results
      for j in filenames:
          tempOutputPath = outputFile+only_skeleton_name+"/"+j[0:-3]+" Combined.mp4"
          #produce superImpose Clip
          result_clip = mpe.VideoFileClip(result_paths[i]+"/"+j, target_resolution=(1080,1920))

          overlay_clip = mpe.VideoFileClip(new_skeleton_path, has_mask=True, target_resolution=(1080,1920))
          masked_clip = mpe.vfx.mask_color(overlay_clip,color=[0,0,0])

          superImpose_clip = mpe.CompositeVideoClip([result_clip,masked_clip])
          combined_clip = mpe.clips_array([[result_clip,overlay_clip,superImpose_clip]])
          combined_clip = combined_clip.resize((1680,560))
          combined_clip.write_videofile(
              tempOutputPath,
              remove_temp=True,
              codec="libx264",
              audio_codec="aac",
              threads=6
          )

          all_Results.append(tempOutputPath)

      #delete Temporary files
      %rm {edited_video_path}
      %rm {new_skeleton_path}

  #display results
  for i in all_Results:
      #Formatting
      grid = widgets.GridspecLayout(2,1) #(i,j) i= number of columns, j= number of rows

      container = widgets.VBox([grid], layout = widgets.Layout(
          height = '650px',
          width='1690px',
          border='3px solid black',
          justify_content = 'center',
          align_items='center'
      ))

      #show skeleton video
      out = widgets.Output()
      with out:
          display(Video(i,embed=True,html_attributes="loop autoplay"))
      grid[0,0] = out

      #display prompt
      out = widgets.Output()
      with out:
          display(HTML("<center><h1>"+i[0:-14].split("/")[len(i[0:-14].split("/"))-1]+"</h1></center>"))
      grid[1,:] = out

      display(container)

run_inference_button.on_click(run_inference)

Moviepy - Building video ./output/samples/vis_ikun_pose2/Iron man on the beach. Combined.mp4.
Moviepy - Writing video ./output/samples/vis_ikun_pose2/Iron man on the beach. Combined.mp4



t:  98%|█████████▊| 56/57 [00:45<00:00,  1.34it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file ./Dummy1.mp4, 8294400 bytes wanted but 0 bytes read,at frame 55/56, at time 0.93/0.93 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready ./output/samples/vis_ikun_pose2/Iron man on the beach. Combined.mp4
